In [1]:
 print('Hello World!')

Hello World!


In [2]:
def say_hello(recipient):
    return 'Hello, {}!'.format(recipient)
say_hello('Tim')

'Hello, Tim!'

# This is a level 1 heading

## This is a level 2 heading

This is some plain text that forms a paragraph. Add emphasis via **bold** or __bold__, and *italic* or _italic_.

Paragraphs must be separated by an empty line.

* Sometimes we want to include lists.
* Which can be bulleted using asterisks.
